In [10]:
"""
LOAD THE DATA
"""
import os
import numpy as np
from parameters import FileParams as fp, DEFAULT_AUDIO_PARAMS as ap

def load_data(dataset: str):
    x = np.load(os.path.join(fp.data_dir, f'{dataset}_x.npy'))
    y = np.load(os.path.join(fp.data_dir, f'{dataset}_y.npy'))
    return x, y

train_x, train_y = load_data('train')
val_x, val_y = load_data('val')

print(f'Percent of train data positive: {np.mean(train_y) * 100:.2f}%')
print('train_x size: ', train_x.nbytes / 1024 / 1024, 'MB')
print(f'Percent of val data positive: {np.mean(val_y) * 100:.2f}%')
print('val_x size: ', val_x.nbytes / 1024 / 1024, 'MB')

Percent of train data positive: 25.14%
train_x size:  1716.052963256836 MB
Percent of val data positive: 0.05%
val_x size:  53.98780822753906 MB


In [54]:
""" 
CREATE THE MODEL

Model architecture copied from
https://www.tensorflow.org/tutorials/audio/simple_audio
"""

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from keras.losses import binary_crossentropy

INPUT_SHAPE = (ap.n_features, ap.n_mfcc)

# Instantiate the `tf.keras.layers.Normalization` layer.
norm_layer = layers.Normalization()
model = models.Sequential([
    layers.Input(shape=INPUT_SHAPE),
    norm_layer,
    layers.Conv1D(32, 3, activation='relu'),
    layers.Conv1D(64, 3, activation='relu'),
    layers.MaxPooling1D(),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid'),
], name='cnn_model')

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=binary_crossentropy,
    metrics=['accuracy'],
)

In [61]:
"""
SET UP FOR TRAINING
"""

from keras.callbacks import TensorBoard, ModelCheckpoint

BATCH_SIZE = 128
EPOCHS = 2
MODEL_NAME = 'simple_cnn'

checkpoint_dir = f'./checkpoints/{MODEL_NAME}'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_dir,
    save_weights_only=False,  # Save the whole model
    monitor='val_accuracy',
    mode='auto',
    save_best_only=True)

log_dir = os.path.join(fp.log_dir, MODEL_NAME)
callbacks = [TensorBoard(log_dir), model_checkpoint_callback]
# Print out info about the model
model.summary()

Model: "cnn_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_4 (Normalizat  (None, 29, 13)           27        
 ion)                                                            
                                                                 
 conv1d_6 (Conv1D)           (None, 27, 32)            1280      
                                                                 
 conv1d_7 (Conv1D)           (None, 25, 64)            6208      
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 12, 64)           0         
 1D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 12, 64)            0         
                                                                 
 flatten_3 (Flatten)         (None, 768)               0 

In [62]:
"""
TRAIN THE MODEL
"""
model.fit(
    train_x,
    train_y,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(val_x, val_y),
    callbacks=callbacks
)

Epoch 1/2
4656/4662 [============================>.] - ETA: 0s - loss: 0.0097 - accuracy: 0.9965

INFO:tensorflow:Assets written to: ./checkpoints\simple_cnn\assets


INFO:tensorflow:Assets written to: ./checkpoints\simple_cnn\assets


4662/4662 [==============================] - 13s 3ms/step - loss: 0.0097 - accuracy: 0.9966 - val_loss: 8.7404e-04 - val_accuracy: 0.9997
Epoch 2/2
4656/4662 [============================>.] - ETA: 0s - loss: 0.0017 - accuracy: 0.9995

INFO:tensorflow:Assets written to: ./checkpoints\simple_cnn\assets


INFO:tensorflow:Assets written to: ./checkpoints\simple_cnn\assets


4662/4662 [==============================] - 14s 3ms/step - loss: 0.0017 - accuracy: 0.9995 - val_loss: 4.7880e-04 - val_accuracy: 0.9998


In [65]:
"""
Evaluate on the validation set
"""

loss, accuracy = model.evaluate(val_x, val_y)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy*100:.5f}%')

587/587 [==============================] - 1s 2ms/step - loss: 4.7880e-04 - accuracy: 0.9998
Loss: 0.00047879500198177993
Accuracy: 99.98402%


In [68]:
"""
USE THE MODEL TO MAKE PREDICTIONS
"""

predictions = model.predict(train_x[:10]).flatten()

for i in range(len(predictions)):
    print(f'Predicted: {predictions[i]:<.4f}, Actual: {train_y[i]}, {"Correct!" if round(predictions[i]) == train_y[i] else "Incorrect"}')

1/1 [==============================] - 0s 12ms/step
Predicted: 1.0000, Actual: 1.0, Correct!
Predicted: 1.0000, Actual: 1.0, Correct!
Predicted: 0.0000, Actual: 0.0, Correct!
Predicted: 1.0000, Actual: 1.0, Correct!
Predicted: 0.0000, Actual: 0.0, Correct!
Predicted: 1.0000, Actual: 1.0, Correct!
Predicted: 1.0000, Actual: 1.0, Correct!
Predicted: 1.0000, Actual: 1.0, Correct!
Predicted: 0.0000, Actual: 0.0, Correct!
Predicted: 0.0000, Actual: 0.0, Correct!


In [67]:
"""
TEST SMALLER MODEL
"""

SMALLER_NAME = 'smaller_cnn'

# Same except half as many filters in the Conv1D layers
norm_layer = layers.Normalization()
smaller_model = models.Sequential([
    layers.Input(shape=INPUT_SHAPE),
    norm_layer,
    layers.Conv1D(16, 3, activation='relu'),
    layers.Conv1D(32, 3, activation='relu'),
    layers.MaxPooling1D(),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid'),
], name='smaller_cnn')

smaller_model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.binary_crossentropy,
    metrics=['accuracy'],
)

smaller_checkpoint_dir = f'./checkpoints/smaller_{SMALLER_NAME}'
smaller_model_checkpoint_callback = ModelCheckpoint(
    filepath=smaller_checkpoint_dir,
    save_weights_only=False,  # Save the whole model
    monitor='val_accuracy',
    mode='auto',
    save_best_only=True)
smaller_callbacks = [
    TensorBoard(os.path.join(fp.log_dir, SMALLER_NAME)),
    model_checkpoint_callback
]
smaller_model.summary()

Model: "smaller_cnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_8 (Normalizat  (None, 29, 13)           27        
 ion)                                                            
                                                                 
 conv1d_14 (Conv1D)          (None, 27, 16)            640       
                                                                 
 conv1d_15 (Conv1D)          (None, 25, 32)            1568      
                                                                 
 max_pooling1d_7 (MaxPooling  (None, 12, 32)           0         
 1D)                                                             
                                                                 
 dropout_14 (Dropout)        (None, 12, 32)            0         
                                                                 
 flatten_7 (Flatten)         (None, 384)               

In [60]:
"""
TRAIN THE SMALLER MODEL
"""
smaller_model.fit(
    train_x,
    train_y,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(val_x, val_y),
    callbacks=smaller_callbacks
)

Epoch 1/2
4662/4662 [==============================] - 9s 2ms/step - loss: 0.0180 - accuracy: 0.9934 - val_loss: 5.8405e-04 - val_accuracy: 0.9998
Epoch 2/2
4662/4662 [==============================] - 8s 2ms/step - loss: 0.0027 - accuracy: 0.9991 - val_loss: 0.0016 - val_accuracy: 0.9997
